In [1]:
import numpy as np
from torchvision.models.resnet import resnet18
from torchvision.models.resnet import resnet34
from torchvision.models.resnet import resnet50
from torchvision.models.resnet import resnet101
from torchvision.models.resnet import resnet152

import torchvision.transforms as transforms
from PIL import Image
import torch

# 'resnet18': 'https://download.pytorch.org/models/resnet18-5c106cde.pth',
# 'resnet34': 'https://download.pytorch.org/models/resnet34-333f7ec4.pth',
# 'resnet50': 'https://download.pytorch.org/models/resnet50-19c8e357.pth',
# 'resnet101': 'https://download.pytorch.org/models/resnet101-5d3b4d8f.pth',
# 'resnet152': 'https://download.pytorch.org/models/resnet152-b121ed2d.pth',


# Resize and normalize input

Calculate normalization parameters ( see https://github.com/ngeorgis/pytorch_onnx_openvino/issues/1 )

In [2]:
cifar_mean   = (0.4914, 0.4822, 0.4465)
cifar_std    = (0.2023, 0.1994, 0.2010)

In [3]:
# use this based on 
# https://pytorch.org/docs/stable/torchvision/models.html
# and https://github.com/pytorch/examples/blob/42e5b996718797e45c46a25c55b031e6768f8440/imagenet/main.py#L89-L101
# also see https://github.com/pytorch/vision/issues/39

imagenet_mean = (0.485, 0.456, 0.406)
imagenet_std  = (0.229, 0.224, 0.225)

In [4]:
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(imagenet_mean, imagenet_std),
])

For CIFAR10 use this scale and mean for OpenVino model optimizer

In [5]:
print("--scale_values=[",cifar_std[0]*255,",",cifar_std[1]*255,",",cifar_std[2]*255,"] ",
      "--mean_values=[",cifar_mean[0]*255,",",cifar_mean[1]*255,",",cifar_mean[2]*255,"]", sep="")

--scale_values=[51.5865,50.847,51.255] --mean_values=[125.307,122.961,113.8575]


For IMAGENET use this scale and mean for OpenVino model optimizer

In [6]:
print("--scale_values=[",imagenet_std[0]*255,",",imagenet_std[1]*255,",",imagenet_std[2]*255,"] ",
      "--mean_values=[",imagenet_mean[0]*255,",",imagenet_mean[1]*255,",",imagenet_mean[2]*255,"]", sep="")

--scale_values=[58.395,57.120000000000005,57.375] --mean_values=[123.675,116.28,103.53]


In [7]:

transform_test_IMAGENET = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
])

In [8]:
transform_test = transform_test_IMAGENET

# First test ResNet50 in PyTorch

In [9]:
net18 = resnet18(pretrained=True)
net34 = resnet34(pretrained=True)
net50 = resnet50(pretrained=True)
net101 = resnet101(pretrained=True)
net152 = resnet152(pretrained=True)

# set test mode for inference
net18.eval()
net34.eval()
net50.eval()
net101.eval()
net152.eval()

im = Image.open('dog.jpeg')
x = transform_test(im)
x = x.unsqueeze(dim=0)

![alt text](dog.jpeg "Dog test image")

# Save the input vector so that we can use it for OpenVino tests

In [10]:
print (x.shape)
np.save("test_in_vector", x)

!ls -l test_in_vector.npy

torch.Size([1, 3, 224, 224])
-rw-rw-r-- 1 ngeorgis ngeorgis 602240 Jan 27 18:49 test_in_vector.npy


# Run inference 

In [11]:
res50 = net50(x)[0]
res50.shape

torch.Size([1000])

In [12]:
print ("First ten output values: ", net50(x)[0][0:10])

First ten output values:  tensor([ 1.4262, -2.8766, -2.2015, -1.2191, -3.0898, -1.4179, -4.3596,  0.9693,
         0.8332,  0.4352], grad_fn=<SliceBackward>)


In [13]:
values, indices = res50.max(0)
print (indices)

tensor(235)


In [14]:
values, indices = torch.topk(res50, 10)
print (values)
print (indices)

tensor([14.0807, 11.5884, 11.1329,  9.3868,  8.8371,  8.8061,  8.4025,  7.9561,
         7.9554,  7.9356], grad_fn=<TopkBackward>)
tensor([235, 225, 255, 275, 160, 174, 224, 244, 170, 227])


In [15]:
number_top = 10

In [16]:
labels ="test_model.labels"
if labels:
    with open(labels, 'r') as f:
        labels_map = [x.split(sep=' ', maxsplit=1)[-1].strip() for x in f]
else:
    labels_map = None

print (labels_map)

['n01440764_tench,_Tinca_tinca', 'n01443537_goldfish,_Carassius_auratus', 'n01484850_great_white_shark,_white_shark,_man-eater,_man-eating_shark,_Carcharodon_carcharias', 'n01491361_tiger_shark,_Galeocerdo_cuvieri', 'n01494475_hammerhead,_hammerhead_shark', 'n01496331_electric_ray,_crampfish,_numbfish,_torpedo', 'n01498041_stingray', 'n01514668_cock', 'n01514859_hen', 'n01518878_ostrich,_Struthio_camelus', 'n01530575_brambling,_Fringilla_montifringilla', 'n01531178_goldfinch,_Carduelis_carduelis', 'n01532829_house_finch,_linnet,_Carpodacus_mexicanus', 'n01534433_junco,_snowbird', 'n01537544_indigo_bunting,_indigo_finch,_indigo_bird,_Passerina_cyanea', 'n01558993_robin,_American_robin,_Turdus_migratorius', 'n01560419_bulbul', 'n01580077_jay', 'n01582220_magpie', 'n01592084_chickadee', 'n01601694_water_ouzel,_dipper', 'n01608432_kite', 'n01614925_bald_eagle,_American_eagle,_Haliaeetus_leucocephalus', 'n01616318_vulture', 'n01622779_great_grey_owl,_great_gray_owl,_Strix_nebulosa', 'n01629

In [17]:
print("Top {} results:\n===============\n".format(number_top))
for i, probs in enumerate(res50):
    if (i<number_top):
        print( labels_map[indices[i]], "\t\t", values[i], indices[i])

Top 10 results:

n02106662_German_shepherd,_German_shepherd_dog,_German_police_dog,_alsatian 		 tensor(14.0807, grad_fn=<SelectBackward>) tensor(235)
n02105162_malinois 		 tensor(11.5884, grad_fn=<SelectBackward>) tensor(225)
n02111129_Leonberg 		 tensor(11.1329, grad_fn=<SelectBackward>) tensor(255)
n02116738_African_hunting_dog,_hyena_dog,_Cape_hunting_dog,_Lycaon_pictus 		 tensor(9.3868, grad_fn=<SelectBackward>) tensor(275)
n02088094_Afghan_hound,_Afghan 		 tensor(8.8371, grad_fn=<SelectBackward>) tensor(160)
n02091467_Norwegian_elkhound,_elkhound 		 tensor(8.8061, grad_fn=<SelectBackward>) tensor(174)
n02105056_groenendael 		 tensor(8.4025, grad_fn=<SelectBackward>) tensor(224)
n02108551_Tibetan_mastiff 		 tensor(7.9561, grad_fn=<SelectBackward>) tensor(244)
n02090721_Irish_wolfhound 		 tensor(7.9554, grad_fn=<SelectBackward>) tensor(170)
n02105412_kelpie 		 tensor(7.9356, grad_fn=<SelectBackward>) tensor(227)


# Convert PyTorch to ONNX

In [18]:
!pip3 install onnxruntime
!pip3 install onnxruntime-gpu

  Using cached https://files.pythonhosted.org/packages/e6/97/39c630134268a29a7c26f5f1c8fd2f7ff089ccee567cb076087ddf1cb6e0/onnxruntime-0.1.4-cp35-cp35m-manylinux1_x86_64.whl
You are using pip version 8.1.1, however version 19.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
  Using cached https://files.pythonhosted.org/packages/17/cb/0def5a44db45c6d38d95387f20057905ce2dd4fad35c0d43ee4b1cebbb19/onnxruntime_gpu-0.1.3-cp35-cp35m-manylinux1_x86_64.whl
You are using pip version 8.1.1, however version 19.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [19]:
torch.onnx._export(net18, x, 'resnet18.onnx', export_params=True)
torch.onnx._export(net34, x, 'resnet34.onnx', export_params=True)
torch.onnx._export(net50, x, 'resnet50.onnx', export_params=True)
torch.onnx._export(net101, x, 'resnet101.onnx', export_params=True)
torch.onnx._export(net152, x, 'resnet152.onnx', export_params=True)

tensor([[ 1.2213e+00, -2.7040e+00, -1.8909e-01, -6.7603e-01, -3.3056e+00,
         -1.9360e+00, -5.2987e+00, -8.9872e-01, -4.4930e-02, -3.9703e-01,
         -1.4685e+00, -2.2869e+00, -3.0948e+00, -3.4699e+00, -1.9134e+00,
         -2.8971e+00,  2.7806e-01, -2.2986e-01, -2.3246e+00, -1.7495e+00,
         -3.4520e+00, -2.4936e-01, -3.3083e+00,  3.5076e-01, -2.8486e+00,
          4.9069e-01, -1.7008e+00, -1.8001e+00, -1.0999e+00, -1.2797e+00,
         -2.8170e+00, -1.7570e+00, -1.0449e+00, -1.6761e+00, -1.7932e-01,
         -2.4822e+00, -2.2645e+00, -2.3596e+00, -1.2436e+00, -2.1854e+00,
         -8.6019e-01, -2.0205e+00, -3.6509e+00, -1.6989e+00, -7.4498e-01,
         -1.4624e-01, -2.6636e+00, -7.3598e-01, -4.0440e+00, -1.6583e+00,
         -1.0470e+00,  3.8976e-01,  2.1675e-01, -1.8390e+00, -6.5013e-01,
         -1.3384e+00, -3.0597e+00, -2.1036e+00, -2.3777e+00,  8.4964e-01,
          7.4710e-01, -1.4211e-01,  1.0217e+00, -3.8827e-02, -6.4364e-01,
         -1.3596e+00, -1.0893e+00, -2.

In [20]:
!ls -lrSh resnet*.onnx 

-rw-rw-r-- 1 ngeorgis ngeorgis  45M Jan 27 18:49 resnet18.onnx
-rw-rw-r-- 1 ngeorgis ngeorgis  84M Jan 27 18:49 resnet34.onnx
-rw-rw-r-- 1 ngeorgis ngeorgis  99M Jan 27 18:49 resnet50.onnx
-rw-rw-r-- 1 ngeorgis ngeorgis 172M Jan 27 18:49 resnet101.onnx
-rw-rw-r-- 1 ngeorgis ngeorgis 233M Jan 27 18:49 resnet152.onnx


# Install the Intel OpenVino SDK

https://software.intel.com/en-us/openvino-toolkit